# Monkeypox

Das Skript bereitet Daten für den Artikel zu Affenpocken vor. Gegenwärtig noch nicht automatisiert. <br>
LD-Artikel: https://edit.nzz.ch/p/nzz/articles/1685302/edit/canvas

Quelle Github: https://github.com/globaldothealth/monkeypox


In [1]:
import pandas as pd
import webbrowser
import pycountry
import gettext
from datetime import datetime
import os

In [2]:
german = gettext.translation('iso3166', pycountry.LOCALES_DIR, languages=['de'])
german.install()

In [3]:
# Pycountry shoud be on latest version, i.e. '22.3.5'. Else, flags might not work.
pycountry.__version__

'22.3.5'

## DF einlesen

In [17]:
# read google sheet
sheet_id = '1CEBhao3rMe-qtCbAgJTn5ZKQMRFWeAeaiXFpBY3gbHE'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv"
df_roh = pd.read_csv(url)

In [21]:
# Pivot wider
df = df_roh.groupby(['Country','Status'])['ID'].count().unstack().reset_index()
df= df.rename(columns={'confirmed':'Bestätigt', 'suspected':'Verdacht'})

# add totals column
df['Total']=df.iloc[:,1:].sum(axis=True)
df=df.sort_values('Total', ascending=False)

# format integers
df = df.fillna(0)
df.iloc[:,1:] = df.iloc[:,1:].astype(int)

# replace Country names our worldmap ids
df['Country'] = df['Country'].str.replace('England', 'United Kingdom')

### Deutsche Ländernamen, Flaggen

In [29]:
def get_german_name(name):
    country = pycountry.countries.get(name=name)
    return _(country.name)

def get_flag(name):
    country = pycountry.countries.get(name=name)
    return country.flag


In [30]:
df['Land'] = df['Country'].apply(get_german_name)
df['Flagge'] =  df['Country'].apply(get_flag)

df = df[['Country', 'Land', 'Flagge','Bestätigt', 'Verdacht', 'Total']]

In [ ]:
# In casse Country-Matching needs debugging
#for name in df['Country'].to_list():
#    country = pycountry.countries.get(name=name)
#    print(country)
#    print(country.flag)

In [31]:
df

Status,Country,Land,Flagge,Bestätigt,Verdacht,Total
14,Spain,Spanien,🇪🇸,41,59,100
13,Portugal,Portugal,🇵🇹,37,0,37
4,Canada,Kanada,🇨🇦,5,18,23
6,United Kingdom,Vereinigtes Königreich,🇬🇧,20,0,20
3,Belgium,Belgien,🇧🇪,3,1,4
11,Italy,Italien,🇮🇹,4,0,4
17,United States,Vereinigte Staaten,🇺🇸,2,1,3
8,Germany,Deutschland,🇩🇪,3,0,3
1,Australia,Australien,🇦🇺,2,0,2
10,Israel,Israel,🇮🇱,1,0,2


## Vergleichen, Version abspeichern

In [32]:
past_csvs= os.listdir('data')

In [35]:
past_csvs.sort()
past_csvs

['.DS_Store', '20220523-15h37.csv', '20220523-15h43.csv', '20220523-16h21.csv']

In [36]:
df_last_version = pd.read_csv('data/'+past_csvs[-1])

In [37]:
print('old == new:', str(df['Total'].sum() == df_last_version['Total'].sum()))
print('new total:', str(df['Total'].sum()))
print('old total:', str(df_last_version['Total'].sum()))

old == new: False
new total: 207
old total: 206


In [38]:
# save most recent data
now = datetime.now()
time_now = now.strftime("%Y%m%d-%Hh%M")

df.to_csv('data/'+ time_now+'.csv', index=False)

## Export für Weltkarte

In [39]:
ids = pd.read_csv('country_ids.csv')

df_worldmap =df.copy()
df_worldmap['Country'] = df_worldmap['Country'].str.replace('United States','United States of America')

df_worldmap = df_worldmap.rename(columns ={'Country':'ID', 'Total':'Wert'})

# merge df with ids
df_worldmap = ids.merge(df_worldmap[['ID', 'Wert']], how='left').sort_values('Wert', ascending=False)

In [44]:
# Check if all countries were recognised - is True if ok
print('Sum Worldmap:', str(df_worldmap['Wert'].sum()))
print('Corresponds with sum df:', str(df_worldmap['Wert'].sum() == df['Total'].sum()))

Sum Worldmap: 207.0
Corresponds with sum df: True


In [45]:
# copy to q-element
df_worldmap=df_worldmap.sort_values('ID', key=lambda col: col.str.lower())

df_worldmap.to_clipboard(index=False)
webbrowser.open('https://q.st.nzz.ch/editor/choropleth/4acf1a0fd4dd89aef4abaeefd0b6f4dc', new=2)

True

## Export für Tabelle

https://q.st.nzz.ch/editor/table/4acf1a0fd4dd89aef4abaeefd0da5ac6

In [ ]:
df[['Land', 'Flagge', 'Bestätigt', 'Verdacht','Total']].to_clipboard(index=False)

webbrowser.open('https://q.st.nzz.ch/editor/table/4acf1a0fd4dd89aef4abaeefd0da5ac6')

## Total für LD-Artikel

In [ ]:
df['Total'].sum()